In [ ]:
import requests
import json

"""NOTE: This code can only run if provided the BDPA list of businesses data set which is named 'df4' in this file to represent the list of businesses within district 4."""

#use yelp api to get more informed business types
api_key = 2
headers = {'Authorization': 'Bearer %s' % api_key}

def return_parent_category_yelp(business_name, latitude, longitude, address, row):
    #make a request to yelp_api business/search to get the sub category alias for a business
    business_url = "https://api.yelp.com/v3/businesses/search"
    params = {'locale' : 'en_US', 'term':business_name, 'latitude': latitude, 'longitude':longitude, 'location':address}
    business_search_req = requests.get(business_url, params=params, headers=headers)
    business_search_res = json.loads(business_search_req.text)

    if len(business_search_res["businesses"]) == 0:  #case where business is not on yelp and no results are returned
        return (row["NAICS_2017_2digit_desc"], row["NAICS_2017_2digit_desc"])
    
    #case where the businesiness isn't on yelp but it returns a similar business or there is no associated category available
    if business_search_res["businesses"][0]["location"]["address1"] != address or len(business_search_res["businesses"][0]["categories"]) == 0: 
        return (row["NAICS_2017_2digit_desc"], row["NAICS_2017_2digit_desc"])

    sub_category = business_search_res["businesses"][0]["categories"][0]["alias"]

    #make a request to the categories endpoint to get the parent category
    category_url = "https://api.yelp.com/v3/categories/" + sub_category
    params = {'locale' : 'en_US'}
    category_req = requests.get(category_url, params=params, headers=headers)
    category_res = json.loads(category_req.text)


    if len(category_res["category"]["parent_aliases"]) == 0: #case where there is not parent category associated with the business
        return (row["NAICS_2017_2digit_desc"], row["NAICS_2017_2digit_desc"])


    parent_category = category_res["category"]["parent_aliases"][0]

    return (sub_category, parent_category)

#return_parent_category_yelp("go forward inc", '42.349978', '-71.164422', "559 Washington St", [])
df4["yelp_business_type"] = df4.apply(lambda x : return_parent_category_yelp(x["business_name"], x["latitude"], x["longitude"], x["street_address"], x)[1], axis=1)
